In [1]:
import numpy as np
import pandas

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import csv

In [2]:
from uci_datasets import Dataset

In [3]:
data_name = "protein"
data = Dataset(data_name)

housing dataset, N=506, d=13


In [4]:
X = data.x
Y = data.y
#X = data[:,:-1]
#Y = data[:, -1].reshape(-1,1)

print(X)
print(Y)

ind = np.arange(X.shape[0])
print(ind)
#np.random.seed(666)
#np.random.shuffle(ind)
prop = 0.9
n = int(X.shape[0] * prop)

[[ -3.4688  -11.364    -1.1268  ...  -0.65553  34.826     0.95694]
 [ -3.5078  -11.364    16.603   ...   1.6445   33.436     5.4169 ]
 [ -3.5633   23.636    -5.0768  ...  -1.5555   37.346    -0.22306]
 ...
 [ -3.5569  -11.364    -7.7268  ...  -0.65553  40.226    -7.1531 ]
 [  3.4091  -11.364     6.9632  ...   1.7445  -36.694     3.0469 ]
 [ -2.3401  -11.364     8.4432  ...  -3.7555  -17.754    -7.1531 ]]
[[ -3.2328  ]
 [ -8.9328  ]
 [ -5.4328  ]
 [-17.533   ]
 [ -4.7328  ]
 [ -6.9328  ]
 [-12.333   ]
 [ 25.767   ]
 [-10.433   ]
 [ -5.1328  ]
 [  1.5672  ]
 [-10.033   ]
 [ -0.33281 ]
 [ -7.2328  ]
 [-14.133   ]
 [  1.0672  ]
 [ -0.032806]
 [ -3.1328  ]
 [  2.0672  ]
 [ 10.567   ]
 [  7.2672  ]
 [ -1.3328  ]
 [ -1.0328  ]
 [-14.033   ]
 [ -7.3328  ]
 [ -2.4328  ]
 [ -5.0328  ]
 [ 10.167   ]
 [  0.76719 ]
 [-14.233   ]
 [ -6.9328  ]
 [ -3.6328  ]
 [ -3.7328  ]
 [ -8.0328  ]
 [ -9.4328  ]
 [ -7.1328  ]
 [ -9.5328  ]
 [ -0.83281 ]
 [ -0.43281 ]
 [ -8.4328  ]
 [ -4.7328  ]
 [  9.1672  ]
 [ -

In [5]:
import torch
import gpytorch as gp
from gpytorch_lattice_kernel import RBFLattice

In [6]:
class SimplexGPModel(gp.models.ExactGP):
  def __init__(self, train_x, train_y):
    likelihood = gp.likelihoods.GaussianLikelihood()
    super().__init__(train_x, train_y, likelihood)

    self.mean_module = gp.means.ConstantMean()
    self.covar_module = gp.kernels.ScaleKernel(
       RBFLattice(ard_num_dims=train_x.size(-1), order=1)
    )

  def forward(self, x):
    mean_x = self.mean_module(x)
    covar_x = self.covar_module(x)
    return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [7]:
ITERATIONS = 100

SGPRRMSE = np.zeros(20)
SGPRRMedSE = np.zeros(20)
SGPRTestLL = np.zeros(20)

for i in range(20):
    print('Split', i + 1, 'of', 20)
    np.random.seed(666 + i) # The devil's seed
    np.random.shuffle(ind)
    
    ### GET TRAINING SPLIT AND PREPROCESS ###
    
    Xtrain = X[ind[:n], :]; Ytrain = Y[ind[:n], :]
    Xtest = X[ind[n:], :]; Ytest = Y[ind[n:], :]
    
    mi, ma = Xtrain.min(axis=0), Xtrain.max(axis=0)
    mami = ma - mi
    mami = np.where(mami > 1e-6, mami, 1.) # Don't divide by zero
    Xtrain = (Xtrain - mi) / mami
    Xtest = (Xtest - mi) / mami

    mi = 0.01
    ma = 0.99
    mami2 = np.where(mami > 1e-6, mami, 0.)
    Xtrain = Xtrain * (ma - mi) + mi
    Xtest = Xtest * (ma - mi) + mi

    meany, stdy = np.average(Ytrain), np.std(Ytrain)
    Ytrain = (Ytrain - meany) / stdy
    
    ############## SGPR ##################
    
    Xtrain = torch.tensor(Xtrain).to(torch.float32)
    Ytrain = torch.tensor(Ytrain).squeeze().to(torch.float32)
    m = SimplexGPModel(Xtrain, Ytrain)
    
    m.train()
    optimizer = torch.optim.Adam(m.parameters(), lr=0.1)
    mll = gp.mlls.ExactMarginalLogLikelihood(m.likelihood, m)
    
    def train(x, y, model, mll, optim, lanc_iter=100, cg_iter=500, pre_size=100, cg_tol=1e-2):
        m.train()

        optim.zero_grad()

        with gp.settings.cg_tolerance(cg_tol), \
           gp.settings.max_cg_iterations(cg_iter), \
           gp.settings.max_preconditioner_size(pre_size), \
           gp.settings.max_root_decomposition_size(lanc_iter):

            output = model(x)
            loss = -mll(output, y)
            loss.backward()
            optim.step()
        
        return loss.item()

    for step in range(ITERATIONS): 
        l = train(Xtrain, Ytrain, m, mll, optimizer)
        if step % 2 == 0:
            print('Step:', step, 'Loss:', l)

    Xtest = torch.tensor(Xtest).to(torch.float32)
    Ytest = torch.tensor(Ytest).squeeze().to(torch.float32)
    
    m.eval()
    with gp.settings.cg_tolerance(1e-4), \
           gp.settings.max_cg_iterations(1000), \
           gp.settings.max_preconditioner_size(100), \
           gp.settings.max_root_decomposition_size(100):
        pred_y = m(Xtest)
    
    pred = meany + stdy*pred_y.mean
    
    RMSE = (pred - Ytest).pow(2).mean(0).sqrt()
    RMedSE = (pred - Ytest).pow(2).median(0).values.sqrt()
    loglik = torch.distributions.Normal(pred,
        pred_y.variance.add(m.likelihood.noise).sqrt() * stdy).log_prob(Ytest).mean()
    
    print('RMSE:', RMSE.item())
    print('RMedSE:', RMedSE.item())
    print('Test log-likelihood:', loglik.item())
    
    ### Append results
    SGPRRMSE[i] = RMSE.item()
    SGPRRMedSE[i] = RMedSE.item()
    SGPRTestLL[i] = loglik.item()
    
print('Experiment concluded!')

Split 1 of 20
Discretized kernel coeffs: tensor([0.3461, 1.0000, 0.3461])
Discretized kernel deriv coeffs: tensor([0.3461, 1.0000, 0.3461])
Using /home/martinj/.cache/torch_extensions/py38_cu102 as PyTorch extensions root...
Emitting ninja build file /home/martinj/.cache/torch_extensions/py38_cu102/cpu_lattice/build.ninja...
Building extension module cpu_lattice...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module cpu_lattice...


/home/martinj/simplex-gp/lib/python3.8/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


Step: 0 Loss: 1.0890693664550781
Step: 2 Loss: 1.0003623962402344
Step: 4 Loss: 0.9142631888389587
Step: 6 Loss: 0.8340057134628296
Step: 8 Loss: 0.7942389249801636
Step: 10 Loss: 0.7357805967330933
Step: 12 Loss: 0.6673582196235657
Step: 14 Loss: 0.6382626295089722
Step: 16 Loss: 0.5970591306686401
Step: 18 Loss: 0.5821237564086914


/home/martinj/simplex-gp/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/martinj/simplex-gp/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/martinj/simplex-gp/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(


NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.

In [ ]:
print('Dataset:', data_name)
print('Average RMSE:', np.mean(SGPRRMSE), 'Standard deviation:', np.std(SGPRRMSE))
print('Average RMedSE:', np.mean(SGPRRMedSE), 'Standard deviation:', np.std(SGPRRMedSE))
print('Average test-ll:', np.mean(SGPRTestLL), 'Standard deviation:', np.std(SGPRTestLL))

In [ ]:
m.eval()
print(m.likelihood.noise)